In [1]:
import os
import pandas as pd
from IPython.display import display

In [2]:
%cd ..
from src.utils import load_text
%cd notebooks

/home/rajk/Machine_Learning/DRL-GEC
/home/rajk/Machine_Learning/DRL-GEC/notebooks


# Local function

In [3]:
def gen_conll_report(model_info):
    results = []
    for model_name, (model_path, model_type) in model_info.items():
        dataset_path = os.path.join(model_path, "conll", f"{model_type}.score")
        data = load_text(dataset_path)
        correct, predicted, total = (eval(line.split(": ")[1]) for line in data[-9:-6])
        p, r, f = (eval(line.split(": ")[1]) for line in data[-3:])
        results.append({
            "Model": model_name, 
            "Correct": correct,  
            "Predicted": predicted,  
            "Total Error": total, 
            "Precision": p, 
            "Recall": r, 
            "F-0.5 Score": f,
        })
    conll_df = pd.DataFrame(results)
    highlight_columns = conll_df.columns.to_list()
    highlight_columns.remove("Model")
    display(conll_df.style.highlight_max(subset=highlight_columns, color='lightgreen', axis=0))
    return conll_df

def gen_jfleg_report(model_info):
    results = []
    for model_name, (model_path, model_type) in model_info.items():
        model_results = {"Model": model_name}
        for score_type in ("dev", "test"):
            dataset_path = os.path.join(model_path, "jfleg", f"{model_type}_{score_type}.score")
            data = load_text(dataset_path)
            score_list = eval(data[-1])
            model_results[f"{score_type.title()} Score"] = score_list[0][0]
        results.append(model_results)
    jfleg_df = pd.DataFrame(results)
    display(jfleg_df.style.highlight_max(subset=["Dev Score", "Test Score"], color='lightgreen', axis=0))
    return jfleg_df

## Model finetuned via SL and RL

In [4]:
model_dict = {
    "Pretrain": (os.path.abspath("sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune": (os.path.abspath("pg_logs/finetune_rl_01_11_2022_13:41/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain,907,1651,2533,0.549400,0.358100,0.496300
1,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
2,Pretrain + RL Fine-Tune,929,1369,2500,0.678600,0.371600,0.582400


,Model,Dev Score,Test Score
0,Pretrain,0.512599,0.537839
1,Pretrain + SL Fine-Tune,0.545552,0.591531
2,Pretrain + RL Fine-Tune,0.530688,0.572043


## Model pretained on different amount of synthetic data 

In [5]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain (2M)": (os.path.abspath("sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain (0.5K) + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_03:11:2022_00:32/"), "model-best"),
    "Pretrain (2M) + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain (0.5K),907,1651,2533,0.549400,0.358100,0.496300
1,Pretrain (2M),1036,1790,2619,0.578800,0.395600,0.529700
2,Pretrain (0.5K) + SL Fine-Tune,1279,1979,2793,0.646300,0.457900,0.597200
3,Pretrain (2M) + SL Fine-Tune,1352,2088,2829,0.647500,0.477900,0.604600


,Model,Dev Score,Test Score
0,Pretrain (0.5K),0.512599,0.537839
1,Pretrain (2M),0.520805,0.556833
2,Pretrain (0.5K) + SL Fine-Tune,0.541262,0.586046
3,Pretrain (2M) + SL Fine-Tune,0.546813,0.594995


## Model with different success reward

- 0.9 explore coefficient

In [6]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (100 interval + 0.1 Success)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
    "Pretrain + RL Fine-Tune (100 interval + 0.5 Success)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_17:24/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (100 interval + 0.1 Success),1167,1868,2694,0.624700,0.433200,0.574000
2,Pretrain + RL Fine-Tune (100 interval + 0.5 Success),924,1364,2512,0.677400,0.367800,0.579800


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (100 interval + 0.1 Success),0.538892,0.582487
2,Pretrain + RL Fine-Tune (100 interval + 0.5 Success),0.530445,0.569613


# Model with different explore coefficient

- +0.1 for success

### Model-Best

In [7]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.1 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_14:31/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_13:10/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (0.1 explore),1127,1709,2678,0.659400,0.420800,0.592300
2,Pretrain + RL Fine-Tune (0.5 explore),1122,1707,2636,0.657300,0.425600,0.592800
3,Pretrain + RL Fine-Tune (0.9 explore),1060,1614,2651,0.656800,0.399800,0.582000


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (0.1 explore),0.540174,0.582753
2,Pretrain + RL Fine-Tune (0.5 explore),0.538418,0.582714
3,Pretrain + RL Fine-Tune (0.9 explore),0.534798,0.578188


### Model-Last

In [21]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.1 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_14:31/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (0.1 explore),1258,2016,2759,0.624000,0.456000,0.581200
2,Pretrain + RL Fine-Tune (0.5 explore),1177,1846,2701,0.637600,0.435800,0.583500
3,Pretrain + RL Fine-Tune (0.9 explore),1167,1868,2694,0.624700,0.433200,0.574000


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (0.1 explore),0.545541,0.589471
2,Pretrain + RL Fine-Tune (0.5 explore),0.541086,0.587938
3,Pretrain + RL Fine-Tune (0.9 explore),0.538892,0.582487


# Model with different explore coefficient (new vs old methods)

- +0.1 for success

| Version | Description |
| ------- | ----------- |
| v0      | Exploit = Sample from Candidate Labels with their logits as probs |
| v1      | Exploit = Use logits.argmax() |
| v2      | Exploit = Use candidate_logits.argmax(). Also insert `$KEEP` in the labels |
| v3      | Exploit = Sample from mean(candidate_probs + lev_dist_probs). Also insert `$KEEP` in the labels |
| v4      | v3 with bugfix related with lev_dist |
| v5      | v0 with `$KEEP` label |
| v6      | v5 with `$UNKONWN` label if no candidate_labels |


### Model-Best

In [6]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (v0 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v1 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_06_11_2022_12:24/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v2 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_06_11_2022_15:43/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v3 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_18:49/"), "model-best"),
    "Pretrain + RL Fine-Tune (v4 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_00:14/"), "model-best"),
    "Pretrain + RL Fine-Tune (v5 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_11:41/"), "model-best"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_15:46/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v0 0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_13:10/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v1 0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_05_11_2022_14:39/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v4 0.75-0.25 explore)": (os.path.abspath("pg_logs/finetune_rl_09_11_2022_12:56/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain (0.5K),907,1651,2533,0.549400,0.358100,0.496300
1,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
2,Pretrain + RL Fine-Tune (v0 0.5 explore),1122,1707,2636,0.657300,0.425600,0.592800
3,Pretrain + RL Fine-Tune (v4 0.5 explore),665,917,2376,0.725200,0.279900,0.550100
4,Pretrain + RL Fine-Tune (v5 0.5 explore),623,837,2346,0.744300,0.265600,0.547100
5,Pretrain + RL Fine-Tune (v6 0.5 explore),682,940,2370,0.725500,0.287800,0.556300


,Model,Dev Score,Test Score
0,Pretrain (0.5K),0.512599,0.537839
1,Pretrain + SL Fine-Tune,0.545552,0.591531
2,Pretrain + RL Fine-Tune (v0 0.5 explore),0.538418,0.582714
3,Pretrain + RL Fine-Tune (v4 0.5 explore),0.504864,0.546938
4,Pretrain + RL Fine-Tune (v5 0.5 explore),0.504053,0.535007
5,Pretrain + RL Fine-Tune (v6 0.5 explore),0.515068,0.547007


### Model-Last

In [7]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (v0 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v1 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_06_11_2022_12:24/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v2 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_06_11_2022_15:43/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v3 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_18:49/"), "model-last"),
    "Pretrain + RL Fine-Tune (v4 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_00:14/"), "model-last"),
    "Pretrain + RL Fine-Tune (v5 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_11:41/"), "model-last"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_15:46/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v0 0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v1 0.9 explore)": (os.path.abspath("pg_logs/finetune_rl_05_11_2022_14:39/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v4 0.75-0.25 explore)": (os.path.abspath("pg_logs/finetune_rl_09_11_2022_12:56/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain (0.5K),907,1651,2533,0.549400,0.358100,0.496300
1,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
2,Pretrain + RL Fine-Tune (v0 0.5 explore),1177,1846,2701,0.637600,0.435800,0.583500
3,Pretrain + RL Fine-Tune (v4 0.5 explore),756,1041,2446,0.726200,0.309100,0.571900
4,Pretrain + RL Fine-Tune (v5 0.5 explore),656,894,2347,0.733800,0.279500,0.553800
5,Pretrain + RL Fine-Tune (v6 0.5 explore),804,1124,2463,0.715300,0.326400,0.577700


,Model,Dev Score,Test Score
0,Pretrain (0.5K),0.512599,0.537839
1,Pretrain + SL Fine-Tune,0.545552,0.591531
2,Pretrain + RL Fine-Tune (v0 0.5 explore),0.541086,0.587938
3,Pretrain + RL Fine-Tune (v4 0.5 explore),0.508027,0.551509
4,Pretrain + RL Fine-Tune (v5 0.5 explore),0.505660,0.542503
5,Pretrain + RL Fine-Tune (v6 0.5 explore),0.520494,0.552868


# Model with different false negative rewards

- 0.5 explore coefficient

### Model-Best

In [11]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (-0.1 fn reward)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.0 fn reward)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_13:51/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (-0.1 fn reward),1122,1707,2636,0.657300,0.425600,0.592800
2,Pretrain + RL Fine-Tune (0.0 fn reward),784,1119,2465,0.700600,0.318100,0.564800


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (-0.1 fn reward),0.538418,0.582714
2,Pretrain + RL Fine-Tune (0.0 fn reward),0.512721,0.547989


### Model-Last

In [12]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (-0.1 fn reward)": (os.path.abspath("pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.0 fn reward)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_13:51/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (-0.1 fn reward),1177,1846,2701,0.637600,0.435800,0.583500
2,Pretrain + RL Fine-Tune (0.0 fn reward),970,1439,2575,0.674100,0.376700,0.582200


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (-0.1 fn reward),0.541086,0.587938
2,Pretrain + RL Fine-Tune (0.0 fn reward),0.521098,0.557999


# Model with different update interval

- 0.5 explore coefficient
- 0.0 fn reward

### Model-Best

In [16]:
model_dict = {
    "Pretrain + RL Fine-Tune (50 interval)": (os.path.abspath("pg_logs/finetune_rl_08_11_2022_12:47/"), "model-best"),
    "Pretrain + RL Fine-Tune (100 interval)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_13:51/"), "model-best"),
    "Pretrain + RL Fine-Tune (500 interval)": (os.path.abspath("pg_logs/finetune_rl_08_11_2022_16:39/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + RL Fine-Tune (50 interval),696,997,2385,0.698100,0.291800,0.546100
1,Pretrain + RL Fine-Tune (100 interval),784,1119,2465,0.700600,0.318100,0.564800
2,Pretrain + RL Fine-Tune (500 interval),804,1168,2473,0.688400,0.325100,0.562600


,Model,Dev Score,Test Score
0,Pretrain + RL Fine-Tune (50 interval),0.508417,0.527804
1,Pretrain + RL Fine-Tune (100 interval),0.512721,0.547989
2,Pretrain + RL Fine-Tune (500 interval),0.513188,0.545174


### Model-Last

In [15]:
model_dict = {
    "Pretrain + RL Fine-Tune (50 interval)": (os.path.abspath("pg_logs/finetune_rl_08_11_2022_12:47/"), "model-last"),
    "Pretrain + RL Fine-Tune (100 interval)": (os.path.abspath("pg_logs/finetune_rl_07_11_2022_13:51/"), "model-last"),
    "Pretrain + RL Fine-Tune (500 interval)": (os.path.abspath("pg_logs/finetune_rl_08_11_2022_16:39/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + RL Fine-Tune (50 interval),929,1413,2587,0.657500,0.359100,0.563800
1,Pretrain + RL Fine-Tune (100 interval),970,1439,2575,0.674100,0.376700,0.582200
2,Pretrain + RL Fine-Tune (500 interval),926,1367,2551,0.677400,0.363000,0.577400


,Model,Dev Score,Test Score
0,Pretrain + RL Fine-Tune (50 interval),0.521022,0.558627
1,Pretrain + RL Fine-Tune (100 interval),0.521098,0.557999
2,Pretrain + RL Fine-Tune (500 interval),0.521452,0.550333


# Model with different environment

- 0.5 explore coefficient
- 0.0 fn reward

> Dropout disabled in 3e-6 lr experiment

### Model-Best

In [11]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_15:46/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k)": (os.path.abspath("pg_logs/finetune_rl_11_11_2022_01:04/"), "model-best"),
    "Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k)": (os.path.abspath("pg_logs/finetune_rl_11_11_2022_08:28/"), "model-best"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k),682,940,2370,0.725500,0.287800,0.556300
2,Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k),864,1201,2503,0.719400,0.345200,0.591200
3,Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k),590,821,2297,0.718600,0.256900,0.528600


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k),0.515068,0.547007
2,Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k),0.521312,0.559663
3,Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k),0.499169,0.524901


### Model-Last

In [12]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k)": (os.path.abspath("pg_logs/finetune_rl_10_11_2022_15:46/"), "model-last"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k)": (os.path.abspath("pg_logs/finetune_rl_11_11_2022_01:04/"), "model-last"),
    "Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k)": (os.path.abspath("pg_logs/finetune_rl_11_11_2022_08:28/"), "model-last"),
}

conll_df = gen_conll_report(model_dict)
jfleg_df = gen_jfleg_report(model_dict)

,Model,Correct,Predicted,Total Error,Precision,Recall,F-0.5 Score
0,Pretrain + SL Fine-Tune,1247,1870,2752,0.666800,0.453100,0.609400
1,Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k),804,1124,2463,0.715300,0.326400,0.577700
2,Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k),933,1337,2546,0.697800,0.366500,0.591000
3,Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k),644,904,2369,0.712400,0.271800,0.538000


,Model,Dev Score,Test Score
0,Pretrain + SL Fine-Tune,0.545552,0.591531
1,Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k),0.520494,0.552868
2,Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k),0.522701,0.564486
3,Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k),0.504913,0.535193
